In [ ]:
# 必要なライブラリをインストール
!pip install -U langchain
!pip install -U openai
!pip install -U llama-index
!pip install -U faiss-cpu


In [ ]:
# OpenAI APIキーの読み込み
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# ログレベルの設定
# デバッグログが不要なら実行しなくて良い
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True)

In [ ]:
# インデックスの作成
from llama_index import SimpleDirectoryReader, ServiceContext, GPTSimpleVectorIndex, LLMPredictor
from langchain.chat_models import ChatOpenAI

#data dirからテキストファイルの情報を取得
documents = SimpleDirectoryReader('data/split').load_data()
# gpt-3.5-turbo を指定（現状デフォルトは davinci ）
service_context = ServiceContext.from_defaults(
  llm_predictor=LLMPredictor(llm=ChatOpenAI(model_name="gpt-3.5-turbo")))
# documents をもとに Embeddings API を通信してベクター取得し GPTSimpleVectorIndex を生成
index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)
# save to disk
index_json = 'index_faq.json'
index.save_to_disk(index_json)

In [ ]:
# インデックスをファイルから読み込む
index = GPTSimpleVectorIndex.load_from_disk(index_json)

In [ ]:
# テンプレートの作成
from llama_index.prompts.prompts import RefinePrompt, QuestionAnswerPrompt
QA_PROMPT_TMPL = (
    "私たちは以下の情報をコンテキスト情報として与えます。 \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "あなたはAIとして、この情報をもとに質問を日本語で答えます。回答にはURLを含めてください。前回と同じ回答の場合は同じ回答を行います。: {query_str}\n"
)
qa_prompt = QuestionAnswerPrompt(QA_PROMPT_TMPL)
REFINE_PROMPT = ("元の質問は次のとおりです: {query_str} \n"
    "既存の回答を提供しました: {existing_answer} \n"
    "既存の答えを洗練する機会があります \n"
    "(必要な場合のみ)以下にコンテキストを追加します。 \n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "新しいコンテキストを考慮して、元の答えをより良く洗練して質問に答えてください。\n"
    "コンテキストが役に立たない場合は、元の回答と同じものを返します。回答にはURLを含めてください。")
refine_prompt = RefinePrompt(REFINE_PROMPT)


In [ ]:
# プロンプト
qry = "クライアントがLineでやりとりしたいと言ってくる"
# ベクター検索 + Chat Completion API 実行
response=index.query(qry,
                     text_qa_template=qa_prompt,
                     refine_template=refine_prompt,
                     )
print(response)
